# Logistic regression

logistic regression 是一種2元分類器以回歸的方式去實現，它將所需的問題的答案轉化成0、1，除此之外他會將模型的輸出額外做一次sigmoid將數值線性變換到0~1之間，if $\hat{y}$ > 0.5 return 1 else 0

也就是說模型的輸出value視為機率值，然而邏輯運算無法直接求導，透過特殊設計的成本函數去描述誤差，我們可以用SGD的方式逐漸優化模型找出近似解。

### $$ Cost(\hat{y}, y) = -log(\hat{y}), (y = 1) $$

### $$ Cost(\hat{y}, y) = -log(1-\hat{y}), (y = 0)$$

這樣設計的目的，是為了將誤差轉換成機率的損失函數，log後$ \hat{y} $越接近1則損失越接近0，反之機率越接近0則損失越接近無限大。

此外人類在分類的過程中其實會應用刪去法的概念，也就是說我們會盡可能的預測每一個機率讓機率=0的分類盡可能降低機率，機率=1的分類盡可能提高預測機率，在實務上這樣的成本函數在設計上會複雜許多，因此通常我們只專注調整最大機率是否一致而已，透過訓練模型會自動訓練出特徵與各分類之間的機率關係。

## 以2分法為例:

## $$ Cost(\hat{y}, y) = -[y*log(\hat{y}) + (1-y) * log(1-\hat{y})]$$

當y=1，後項 = 0, $\hat{y}$要盡可能調整=1  
 y=0，前項 = 0, $\hat{y}$要盡可能調整=0

In [1]:
# 以sklearn iris datasets建立測試用資料


from sklearn import datasets

iris = datasets.load_iris()

features = iris['data']
labels = iris['target']


In [2]:
# sklearn implement logistic

from sklearn.linear_model import LogisticRegression
import numpy as np



def MinMax(arr):
    return (arr-arr.min(axis=0))/(arr.max(axis=0)-arr.min(axis=0))

def shuffle(x, y):
    
    indices = np.arange(0, len(y))
    np.random.shuffle(indices)
    return x[indices], y[indices]
    

def acc(yhat, y):
    yhat = yhat.reshape(-1)
    y = y.reshape(-1)
    count = (yhat==y).sum()
    print('%d / %d' %(count, len(y)))
    
train_x = MinMax(features)
train_y = labels==1

train_x, train_y = shuffle(train_x, train_y)

net = LogisticRegression()
net.fit(train_x, train_y)



acc(net.predict(train_x), train_y)
print('label: ', train_y[:10])
print('predict: ', net.predict(train_x[:10]))
print(net.predict_proba(train_x[:10]))

105 / 150
label:  [ True False False  True False False False False  True  True]
predict:  [False False False False False False False False False False]
[[0.75057571 0.24942429]
 [0.64567838 0.35432162]
 [0.82106796 0.17893204]
 [0.54471501 0.45528499]
 [0.64293444 0.35706556]
 [0.71908353 0.28091647]
 [0.7565868  0.2434132 ]
 [0.56243098 0.43756902]
 [0.58575282 0.41424718]
 [0.50384475 0.49615525]]


# Softmax Regression


softmax regression 是一種 logistic regression的廣義應用也就是多元邏輯回歸，當資料類別無法用2元滿足時我們可以用多個2元分類器去實作，也可以用多元分類方式。這2種方式各有利弊，我們假設把一個有5類的類別表達我們可以用5個binary表達 ex: 00100，我們可以用5個binary classifier對每個類別個別檢測，好處是對於一個label有多個類別存在時也能預測，缺點是對於一個label僅一個類別的互斥(winner takes all)關係中，無法有效加入先驗知識進行訓練，因此效果不佳;softmax剛好相反。

softmax regression其實就是在一個以上輸出的logistic regression做一次softmax的動作，**Softmax會把每個輸出轉化為0~1之間且總合為1，數字之間的大小關係不改變**



## $$ Cost(\hat{y}, y) = - \frac{1}{n}\sum_{i=1}^n y_i * log(\hat{y}_i)$$

" i "是每個個別分類機率，當y是one-hot encode時，除了主類別以外的$y_i$值都是0，loss = 主類別的機率loss

而softmax的規律會讓某個數值提高時同時降低其他的數值，就如同機率一般，這樣的特性會在訓練winner take all關係特別容易。


In [3]:
# softmax

train_x = MinMax(features)
train_y = labels

train_x, train_y = shuffle(train_x, train_y)

net = LogisticRegression(multi_class = 'multinomial', solver='lbfgs')
net.fit(train_x, train_y)



acc(net.predict(train_x), train_y)
print('label: ', train_y[:10])
print('predict: ', net.predict(train_x[:10]))
print(net.predict_proba(train_x[:10]))

141 / 150
label:  [0 0 1 1 0 0 0 1 2 0]
predict:  [0 0 1 1 0 0 0 1 2 0]
[[0.85935056 0.13711587 0.00353357]
 [0.87488105 0.12096597 0.00415298]
 [0.03391241 0.64601164 0.32007596]
 [0.09264514 0.59623877 0.31111609]
 [0.8690667  0.12815365 0.00277966]
 [0.93758572 0.06068196 0.00173232]
 [0.87159049 0.12502279 0.00338672]
 [0.13749355 0.60538544 0.25712101]
 [0.04144649 0.41784153 0.54071198]
 [0.87273607 0.1234798  0.00378414]]
